# Creating Sub Agents

## Introduction

In this notebook we show you how to create your first sub-agent on [Amazon Bedrock Agents](https://aws.amazon.com/bedrock/agents/).

Amazon Bedrock Agents enable generative AI applications to execute multi-step business tasks using natural language.

In our first example we will create a Portfolio agent, where customers can ask the agent to return information about Portfolio for certain company stocks. 

The following represents the piece of architecture that will be built on this module.





## Setup

Firstly, you are going to install boto3 dependencies from pip. Make sure you have the latest version of it for full capabilities

In [ ]:
!pip uninstall boto3 botocore awscli --yes

In [2]:
# Install latest boto3
!python3 -m pip install --force-reinstall --no-cache -q --no-dependencies -r ../requirements.txt

#### Restart kernel

If you face issues to apply the latest multi-agent capabilities, uncomment this line to restart kernel to ensure packages updates to take effect

In [3]:
import IPython

#IPython.Application.instance().kernel.do_shutdown(True)

Check your boto3 version

In [ ]:
!pip freeze | grep boto3

## Creating Agent

On this section we declare global variables that will be act as helpers during entire notebook and you will start to create your first agent.

In [5]:
import boto3
import os
import json
import time

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name
account_id_suffix = account_id[:3]
agent_suffix = f"{region}-{account_id_suffix}"

s3_client = boto3.client('s3', region)
bedrock_client = boto3.client('bedrock-runtime', region)

agent_foundation_model = [
    'anthropic.claude-3-5-sonnet-20240620-v1:0',
    'anthropic.claude-3-sonnet-20240229-v1:0',
    'anthropic.claude-3-haiku-20240307-v1:0',
    'amazon.nova-lite-v1:0',
    'amazon.nova-pro-v1:0',
    'amazon.nova-micro-v1:0'
]

In [6]:
portfolio_agent_name = f"portfolio-{agent_suffix}"

portfolio_lambda_name = f"fn-portfolio-agent-{agent_suffix}"

portfolio_agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{portfolio_agent_name}'


### Importing helper functions

On following section, we're adding `bedrock_agent_helper.py` and `knowledge_base_helper` on Python path, so the files can be recognized and their functionalities can be invoked.

Now, you're going to import from helper classes `bedrock_agent_helper.py` and `knowledge_base_helper.py`.
 
Those files contain helper classes totally focused on make labs experience smoothly. 

All interactions with Bedrock will be handled by these classes.

Following are methods that you're going to invoke on this lab:

On `agents.py`:
- `create_agent`: Create a new agent and respective IAM roles
- `add_action_group_with_lambda`: Create a lambda function and add it as an action group for a previous created agent
- `create_agent_alias`: Create an alias for this agent
- `invoke`: Execute agent

On `knowledge_bases.py`:
- `create_or_retrieve_knowledge_base`: Create Knowledge Base on Amazon Bedrock if it doesn't exist or get info about previous created.
- `synchronize_data`: Read files on S3, convert text info into vectors and add that information on Vector Database.

In [7]:
import sys

sys.path.insert(0, ".")
sys.path.insert(1, "..")

from utils.bedrock_agent_helper import (
    AgentsForAmazonBedrock
)
from utils.knowledge_base_helper import (
    KnowledgeBasesForAmazonBedrock
)
agents = AgentsForAmazonBedrock()
kb = KnowledgeBasesForAmazonBedrock()

## Creating Agent

Create the forecast agent that will have an `Amazon Bedrock Knowledge Base` with information on how forecast process is done as well as the `action groups` to handle the user requests.

In order to have accurate agents, it is important to set unambiguous instructions of what the agent should do and what it should not do. It is also important to provide clear definitions for when the agent should use the knowledge bases and action groups available to it.

We will provide the following instructions to our agent:
```
You are an investment analyst. Your job is to assist in investment analysis, create research summaries, generate profitable company portfolios, and facilitate communication through emails. Here is how I want you to think step by step:

1. Portfolio Creation:
    Analyze the user's request to extract key information such as the desired number of companies and industry. 
    Based on the criteria from the request, create a portfolio of companies. Use the template provided to format the portfolio.

2. Company Research and Document Summarization:
    For each company in the portfolio, conduct detailed research to gather relevant financial and operational data.
    When a document, like the FOMC report, is mentioned, retrieve the document and provide a concise summary.

3. Email Communication:
    Using the email template provided, format an email that includes the newly created company portfolio and any summaries of important documents.
    Utilize the provided tools to send an email upon request, That includes a summary of provided responses and portfolios created.

Core behaviors:
1. Always use available information systems before asking customers for additional details
2. Maintain a professional yet conversational tone
3. Provide clear, direct answers without referencing internal systems or data sources
4. Present information in an easy-to-understand manner
5. DO NOT plot graphs. Refuse to do so when asked by the user. Instead provide an overview of the data

Response style:
- Be helpful and solution-oriented
- Use clear, non-technical language
- Focus on providing actionable insights
- Maintain natural conversation flow
- Be concise yet informative 
- do not add extra information not required by the user
```


In [ ]:
agent_description = """You are an investment analyst. Your job is to assist in investment analysis, create research summaries, generate profitable company portfolios, and facilitate communication through emails"""

agent_instruction = """You are an investment analyst. Your job is to assist in investment analysis, create research summaries, generate profitable company portfolios, and facilitate communication through emails. Here is how I want you to think step by step:

1. Portfolio Creation:
    Analyze the user's request to extract key information such as the desired number of companies and industry. 
    Based on the criteria from the request, create a portfolio of companies. Use the template provided to format the portfolio.

2. Company Research and Document Summarization:
    For each company in the portfolio, conduct detailed research to gather relevant financial and operational data.
    When a document, like the FOMC report, is mentioned, retrieve the document and provide a concise summary.

3. Email Communication:
    Using the email template provided, format an email that includes the newly created company portfolio and any summaries of important documents.
    Utilize the provided tools to send an email upon request, That includes a summary of provided responses and portfolios created.
Core behaviors:
1. Always use available information systems before asking customers for additional details
2. Maintain a professional yet conversational tone
3. Provide clear, direct answers without referencing internal systems or data sources
4. Present information in an easy-to-understand manner
5. DO NOT plot graphs. Refuse to do so when asked by the user. Instead provide an overview of the data

Response style:
- Be helpful and solution-oriented
- Use clear, non-technical language
- Focus on providing actionable insights
- Maintain natural conversation flow
- Be concise yet informative 
- do not add extra information not required by the user"""

portfolio_agent = agents.create_agent(
    portfolio_agent_name,
    agent_description,
    agent_instruction,
    agent_foundation_model,
    code_interpretation=True
)

portfolio_agent

### Creating Lambda

In order to enable the agent to execute tasks, we will create an AWS Lambda function that implements the tasks execution. We will then provide this lambda function to the agent action group. You can find more information on how to use action groups to define actions that your agent can perform [here](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-action-create.html)

On this block, we're going to generate Lambda function Code:

In [9]:
import boto3
import json
import os

def get_named_parameter(event, name):
    return next(item for item in event['parameters'] if item['name'] == name)['value']

def companyResearch(event):
    companyName = get_named_parameter(event, 'name').lower()
    print("NAME PRINTED: ", companyName)
    for company_info in company_data:
        if company_info["companyName"].lower() == companyName:
            return company_info
    return None

def createPortfolio(event, company_data):
    numCompanies = int(get_named_parameter(event, 'numCompanies'))
    industry = get_named_parameter(event, 'industry').lower()
    industry_filtered_companies = [company for company in company_data if company['industrySector'].lower() == industry]
    sorted_companies = sorted(industry_filtered_companies, key=lambda x: x['profit'], reverse=True)
    top_companies = sorted_companies[:numCompanies]
    return top_companies

def sendEmail(event, company_data):
    emailAddress = get_named_parameter(event, 'emailAddress')
    fomcSummary = get_named_parameter(event, 'fomcSummary')
    # Retrieve the portfolio data as a string

    portfolioDataString = get_named_parameter(event, 'portfolio')

    # Prepare the email content
    email_subject = "Portfolio Creation Summary and FOMC Search Results"
    email_body = f"FOMC Search Summary:\n{fomcSummary}\n\nPortfolio Details:\n{json.dumps(portfolioDataString, indent=4)}"

    # Email Code is Set here
    CHARSET = "UTF-8"
    response = client.send_email(
            Destination={
                "ToAddresses": [
                    "aswathsr@amazon.com",
                ],
                
                },
            Message={
                "Body": {
                    "Text": {
                        "Charset": CHARSET,
                        "Data": email_body,
                    
                    }
                },
                "Subject": {
                    "Charset": CHARSET,
                    "Data": email_subject,
                
                },
                
            },
            Source="aswathsr@amazon.com",
    )
    
    return "Email sent successfully to {}".format(emailAddress)

def lambda_handler(event, context):
    print(event)
    company_data = [
        #Technology Industry
        {"companyId": 1, "companyName": "TechStashNova Inc.", "industrySector": "Technology", "revenue": 10000, "expenses": 3000, "profit": 7000, "employees": 10},
        {"companyId": 2, "companyName": "QuantumPirateLeap Technologies", "industrySector": "Technology", "revenue": 20000, "expenses": 4000, "profit": 16000, "employees": 10},
        {"companyId": 3, "companyName": "CyberCipherSecure IT", "industrySector": "Technology", "revenue": 30000, "expenses": 5000, "profit": 25000, "employees": 10},
        {"companyId": 4, "companyName": "DigitalMyricalDreams Gaming", "industrySector": "Technology", "revenue": 40000, "expenses": 6000, "profit": 34000, "employees": 10},
        {"companyId": 5, "companyName": "NanoMedNoLand Pharmaceuticals", "industrySector": "Technology", "revenue": 50000, "expenses": 7000, "profit": 43000, "employees": 10},
        {"companyId": 6, "companyName": "RoboSuperBombTech Industries", "industrySector": "Technology", "revenue": 60000, "expenses": 8000, "profit": 52000, "employees": 12},
        {"companyId": 7, "companyName": "FuturePastNet Solutions", "industrySector": "Technology",  "revenue": 60000, "expenses": 9000, "profit": 51000, "employees": 10},
        {"companyId": 8, "companyName": "InnovativeCreativeAI Corp", "industrySector": "Technology", "revenue": 65000, "expenses": 10000, "profit": 55000, "employees": 15},
        {"companyId": 9, "companyName": "EcoLeekoTech Energy", "industrySector": "Technology", "revenue": 70000, "expenses": 11000, "profit": 59000, "employees": 10},
        {"companyId": 10, "companyName": "TechyWealthHealth Systems", "industrySector": "Technology", "revenue": 80000, "expenses": 12000, "profit": 68000, "employees": 10},
    
        #Real Estate Industry
        {"companyId": 11, "companyName": "LuxuryToNiceLiving Real Estate", "industrySector": "Real Estate", "revenue": 90000, "expenses": 13000, "profit": 77000, "employees": 10},
        {"companyId": 12, "companyName": "UrbanTurbanDevelopers Inc.", "industrySector": "Real Estate", "revenue": 100000, "expenses": 14000, "profit": 86000, "employees": 10},
        {"companyId": 13, "companyName": "SkyLowHigh Towers", "industrySector": "Real Estate", "revenue": 110000, "expenses": 15000, "profit": 95000, "employees": 18},
        {"companyId": 14, "companyName": "GreenBrownSpace Properties", "industrySector": "Real Estate", "revenue": 120000, "expenses": 16000, "profit": 104000, "employees": 10},
        {"companyId": 15, "companyName": "ModernFutureHomes Ltd.", "industrySector": "Real Estate", "revenue": 130000, "expenses": 17000, "profit": 113000, "employees": 10},
        {"companyId": 16, "companyName": "CityCountycape Estates", "industrySector": "Real Estate", "revenue": 140000, "expenses": 18000, "profit": 122000, "employees": 10},
        {"companyId": 17, "companyName": "CoastalFocalRealty Group", "industrySector": "Real Estate", "revenue": 150000, "expenses": 19000, "profit": 131000, "employees": 10},
        {"companyId": 18, "companyName": "InnovativeModernLiving Spaces", "industrySector": "Real Estate", "revenue": 160000, "expenses": 20000, "profit": 140000, "employees": 10},
        {"companyId": 19, "companyName": "GlobalRegional Properties Alliance", "industrySector": "Real Estate", "revenue": 170000, "expenses": 21000, "profit": 149000, "employees": 11},
        {"companyId": 20, "companyName": "NextGenPast Residences", "industrySector": "Real Estate", "revenue": 180000, "expenses": 22000, "profit": 158000, "employees": 260}
    ]

    result = ''
    response_code = 200
    action_group = event['actionGroup']
    api_path = event['apiPath']
    
    print("api_path: ", api_path )
    
    if api_path == '/companyResearch':
        result = companyResearch(event)
    elif api_path == '/createPortfolio':
        result = createPortfolio(event, company_data)
    elif api_path == '/sendEmail':
        result = sendEmail(event, company_data)
    else:
        response_code = 404
        result = f"Unrecognized api path: {action_group}::{api_path}"
        
    response_body = {
        'application/json': {
            'body': result
        }
    }
        
    action_response = {
        'actionGroup': event['actionGroup'],
        'apiPath': event['apiPath'],
        'httpMethod': event['httpMethod'],
        'httpStatusCode': response_code,
        'responseBody': response_body
    }

    api_response = {'messageVersion': '1.0', 'response': action_response}
    return api_response

### Defining available actions

Next we will define the available actions that an agent can perform using [Function Details](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-action-function.html). You can also do this task using OpenAPI Schemas, which can be very useful if you already have an OpenAPI schema available for your application.

When creating your function details, it is important to provide clear descriptions for the function and for its parameters, as your agent depends on them to correctly orchestrate the tasks to be executed

In [20]:
functions_def = [
    {
        "name": "companyResearch",
        "description": "Get financial data for a company by name",
        "parameters": {
            "name": {
                "description": "Name of the company to research",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        "name": "createPortfolio",
        "description": "Create a company portfolio of top profit earners by specifying number of companies and industry",
        "parameters": {
            "numCompanies": {
                "description": "Number of companies to include in the portfolio",
                "required": True,
                "type": "integer"
            },
            "industry": {
                "description": "Industry sector for the portfolio companies",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        "name": "sendEMail",
        "description": "Send an email with FOMC search summary and created portfolio",
        "parameters": {
            "emailAddress": {
                "description": "Recipient's email address",
                "required": True,
                "type": "string"
            },
            "fomcSummary": {
                "description": "Summary of FOMC search results",
                "required": True,
                "type": "string"
            },
            "portfolio": {
                "description": "Details of the created stock portfolio (as a JSON string)",
                "required": True,
                "type": "string"  # Changed from "object" to "string"
            }
        }
    }
]

## Schema 

The OpenAPI schema is also provided for the example for better understanding. 


### Creating action group and attaching to the agent
Now it's time to add this Lambda function and the function details as an action group for this agent and prepare it.

In [21]:

agents.add_action_group_with_lambda(
    agent_name=portfolio_agent_name,
    lambda_function_name=portfolio_lambda_name,
    source_code_file="portfolio.py",
    agent_functions=functions_def,
    agent_action_group_name="Portfolio_actions",
    agent_action_group_description="Function to get portfolio and send emails ",
)

## Testing Agent

Now, let's run some tests on the agent we just created to make sure it's working. To do so we will use our test alias: `TSTALIASID` which allows you to invoke a draft version of your agent

### Testing portfolio action
Now we can test the test the portfolio action

In [ ]:
%%time
response = agents.invoke(
    """can you give me the portfolio for all Realestate companies present""", 
    portfolio_agent[0], enable_trace=True
)
print("====================")
print(response)

In [31]:
time.sleep(60)

## Create alias

As you can see, you can use your agent with the `TSTALIASID` to complete tasks. 
However, for multi-agents collaboration it is expected that you first test your agent and only use it once it is fully functional. 
Therefore to use an agent as a sub-agent in a multi-agent collaboration you first need to create an agent alias and connect it to a new version. 

Since we've tested and validated our agent, let's now create an alias for it:

In [32]:
portfolio_agent_agent_alias_id, portfolio_agent_alias_arn = agents.create_agent_alias(
    portfolio_agent[0], 'v1'
)

## Now we have created our portfolio agent, let us create our Data Assistant Agent. 



In this example,. We are also using [Amazon Bedrock Knowledge Bases](https://aws.amazon.com/bedrock/knowledge-bases/) to store data that contains relevant information. This is completely up to the users on what reports they have to store, for easier method we are storing the FOMC reports, and stock related details on to the bucket

#### Creating the Agent

In [33]:
import boto3
import os
import json
import time
import boto3
from datetime import datetime, timedelta
import random
sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name
account_id_suffix = account_id[:3]
agent_suffix = f"{region}-{account_id_suffix}"

s3_client = boto3.client('s3', region)
bedrock_client = boto3.client('bedrock-runtime', region)

agent_foundation_model = [
    'anthropic.claude-3-5-sonnet-20240620-v1:0',
    'anthropic.claude-3-sonnet-20240229-v1:0',
    'anthropic.claude-3-haiku-20240307-v1:0'
]

In [36]:
data_assistant_agent_name = f"data-{agent_suffix}"


data_assistant_agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{data_assistant_agent_name}'


knowledge_base_name = f'{data_assistant_agent_name}-kb'
suffix = f"{region}-{account_id}"

knowledge_base_description = "KB containing information contaning relevant data"
bucket_name = f'data-assistant-kb-{suffix}'

## Create and syncronize Knowledge Base

On this section, you're going to create a Amazon Bedrock Knowledge Base and ingest data on it.

This data contains basic information about how forecast process is done.

**This creation process can take several minutes.**

In [ ]:
%%time
kb_id, ds_id = kb.create_or_retrieve_knowledge_base(
    knowledge_base_name,
    knowledge_base_description,
    bucket_name
)

print(f"Knowledge Base ID: {kb_id}")
print(f"Data Source ID: {ds_id}")

## Upload Data to Load on S3


This data should be uploaded into a S3 bucket and then added into an Amazon Bedrock Knowledge Base. The example data is provided in the repository

In [ ]:
# AWS S3 configuration
S3_BUCKET_NAME = 'data-assistant-kb-{suffix}'  # Replace with your S3 bucket name
AWS_REGION = 'us-east-1'  # Replace with your AWS region

In [ ]:
s3_client = boto3.client('s3')

In [ ]:
LOCAL_FOLDER_PATH = 'Financial-Analyst-Agents/Data'


def upload_files(local_folder_path, S3_BUCKET_NAME):
    """
    Uploads all files from the local folder to the specified S3 bucket.

    Args:
        local_folder_path: The path to the local folder containing the files.
        BUCKET_NAME: The name of the S3 bucket.
    """
    for filename in os.listdir(local_folder_path):
        if os.path.isfile(os.path.join(local_folder_path, filename)):
            # Construct the full path to the local file
            local_file_path = os.path.join(local_folder_path, filename)

            # Upload the file to S3
            try:
                s3_client.upload_file(local_file_path, S3_BUCKET_NAME, filename)
                print(f"Successfully Uploaded {filename} to {S3_BUCKET_NAME}")
            except Exception as e:
                print(f"Error uploading the file {str(e)}")

upload_files(LOCAL_FOLDER_PATH, BUCKET_NAME)

### Synchronizing Knowledge Base
Now that the data is available in the s3 bucket, let's synchronize it to our knowledge base

In [ ]:

# sync knowledge base
kb.synchronize_data(kb_id, ds_id)

In [ ]:
kb_info = kb.get_kb(kb_id)
kb_arn = kb_info['knowledgeBase']['knowledgeBaseArn']

In [ ]:
kb_config = {
    'kb_id': kb_id,
    'kb_instruction': """Use this knowledge base when a user asks about data, such as economic trends, company financial statements, or the outcomes of the Federal Open Market Committee meetings,sp500 and nasdaq"""
}

In [ ]:
agent_description = """You are a Data Assistant Bot that will get contextual and relevant information from the knowledge base regarding the financial data questions that the user will have, """

agent_instruction = """You will answer only from KnowledgeBase and nothing else 

You will help in agent orchestration for providing data if it is about any details asked on fomc or stock data from nasdaq or KB
Your Scope of Support is ONLY restricted to the scope of the knowledge Base
You will ALWAYS provide relevant context and references on trace
"""
data_assistant_agent = agents.create_agent(
    data_assistant_agent_name,
    agent_description,
    agent_instruction,
    agent_foundation_model,
    kb_arns=[kb_arn],
    code_interpretation=True
)

data_assistant_agent

### Associating knowledge base
Now that we've created the agent, let's associate the previously created knowledge base to it.

In [ ]:
agents.associate_kb_with_agent(
    data_assistant_agent[0],
    kb_config['kb_instruction'],
    kb_config['kb_id']
)

## Testing Agent

Now, let's run some tests on the agent we just created to make sure it's working. To do so we will use our test alias: `TSTALIASID` which allows you to invoke a draft version of your agent

In [ ]:
%%time
response = agents.invoke(
    """can you give the minutes of fomc reports present """, 
    data_assistant_agent[0], enable_trace=True
)
print("====================")
print(response)

## Create alias

As you can see, you can use your agent with the `TSTALIASID` to complete tasks. 
However, for multi-agents collaboration it is expected that you first test your agent and only use it once it is fully functional. 
Therefore to use an agent as a sub-agent in a multi-agent collaboration you first need to create an agent alias and connect it to a new version. 

Since we've tested and validated our agent, let's now create an alias for it:

In [ ]:
data_assistant_agent_alias_id, data_assistant_agent_alias_arn = agents.create_agent_alias(
    data_assistant_agent[0], 'v1'
)

## Saving information
Let's store some environment variables to be used on our next notebooks.

In [ ]:
forecast_agent_arn = agents.get_agent_arn_by_name(forecast_agent_name)
data_assistant_agent_id = data_assistant_agent[0]
data_assistant_agent_kb = knowledge_base_name
portfolio_agent_arn = agents.get_agent_arn_by_name(portfolio_agent_name)
portfolio_agent_id = portfolio_agent[0]



%store portfolio_agent_name
%store portfolio_agent_arn
%store portfolio_agent_id
%store portfolio_lambda_name
%store portfolio_agent_agent_alias_id
%store portfolio_agent_agent_alias_arn
%store data_assistant_agent_alias_arn
%store data_assistant_agent_name
%store data_assistant_agent_id
%store data_assistant_agent_kb
